# Imports

In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

import os
import time

from matplotlib import pyplot as plt
from IPython import display
import ntpath
import librosa
import numpy as np
from PIL import Image

In [ ]:
!pip install -q -U tensorboard

In [ ]:
# let's try with MOBIPHONE dataset
!wget --no-check-certificate -r "https://drive.google.com/uc?export=download&id=1MflXkdaeAFyBftKkCTTuUXkfJ-fOTzus" -O "MOBIPHONE.zip"

In [ ]:
!unzip MOBIPHONE.zip
PATH = 'MOBIPHONE/'
folders = ["Apple iPhone 5", "HTC desire c", "HTC Sensation xe", "LG GS290", "LG L3", "LG Optimus L5", "LG Optimus L9", "Nokia 5530", "Nokia C5", "Nokia N70", "Samsung e1230", "Samsung E2121B", "Samsung E2600", "Samsung Galaxy GT-I9100 s2", "Samsung GT-I8190 mini", "Samsung GT-N7100 (galaxy note2)", "Samsung Galaxy Nexus S", "Samsung s5830i", "Sony Ericson c902", "Sony ericson c510i"]
# "Vodafone joy 845" will be used for test, all of the above ones for training

In [ ]:
#to find length of shortest file (317427)

#minimum_length = 900000
#for i in range(len(folders)):
  #for audio in os.listdir(PATH+folders[i]+"/"):
    #ext = os.path.splitext(audio)[-1].lower()
    #if ext == ".wav":
      #y, sr = librosa.load(PATH+folders[i]+"/"+audio)
      #if len(y) < minimum_length:
        #minimum_length = len(y)

minimum_length = 317427

In [ ]:
# now, let's convert audio files into images...
!mkdir frequency_images_train # directory where to save frequency representation images for train
!mkdir frequency_images_test # directory where to save frequency representation images for test

# ask input parameters
#data = input("1-STFT / 2-LOG MEL SPECT\n")
#reconstruction = input("1-Original Phase / 2-Griffinlim\n")
#while (data != "1" and data != "2") or (reconstruction != "1" and reconstruction != "2"):
#    data = input("1-STFT / 2-LOG MEL SPECT\n")
#    reconstruction = input("1-Original Phase / 2-Griffinlim\n")



#phase can be discarded for now...it will just be useful to go back to audio for the testing cases

n_files = 24 # inside test folder
i = 1 # index of output file (progressive)
starting_point=50 # where to start mask
m = 8 # to have a total degradation of 24 rows/columns
data = "2"


#training folders cycle
id=1 #progressive ID, just to rename images
for i in range(len(folders)-1):
  for audio in os.listdir(PATH+folders[i]+"/"):
    ext = os.path.splitext(audio)[-1].lower()
    if ext == ".wav":
      y, sr = librosa.load(PATH+folders[i]+"/"+audio, sr=16000)
      if len(y)>=20*sr-1:
        y = y[0:20*sr]
        if data == "1":
          freq_rep = librosa.stft(y, n_fft=1023, win_length=625, hop_length=625, window='rect')
          ph = np.angle(freq_rep)
        else:
          freq_rep = librosa.stft(y, n_fft=1023, win_length=625, hop_length=625, window='rect')
          ph = np.angle(freq_rep)
          freq_rep = librosa.feature.melspectrogram(S=abs(freq_rep), n_mels=512, sr=16000, power=1.0)

        if data == "1":
          mag_db = librosa.amplitude_to_db(np.abs(freq_rep))
        else:
          mag_db = librosa.amplitude_to_db(freq_rep)  # mel

        # max and min values (in db) for conversion
        max_val = np.max(np.max(mag_db))
        min_val = np.min(np.min(mag_db))
        # converting into image
        freq_rep_img = np.round(((mag_db-min_val)/(max_val-min_val))*(255-0)+0)

        #create also masked image
        mask = np.ones([len(freq_rep_img), len(freq_rep_img[0])])
        mask[starting_point:(starting_point+m+1), :] = 0

        corrupted_img = mask * freq_rep_img

        composed = np.zeros([len(freq_rep_img), len(freq_rep_img[0])*2])
        composed[:,0:len(freq_rep_img[0])] = freq_rep_img
        composed[:,len(freq_rep_img[0]):len(freq_rep_img[0])*2] = corrupted_img

        composed_image = Image.fromarray(composed.astype(np.uint8))  
        composed_image.save("frequency_images_train/speaker_train_"+str(id)+".png")
        id=id+1

        #IDEA: 
        # -build one big image composed by original|corrupted
        # -save it
        # -decompose it in load() function...as it was done for original Pix2Pix

#testing folder
m = 8 # just to start filling it
id=1
for audio in sorted(os.listdir(PATH+"Vodafone joy 845/")):
  ext = os.path.splitext(audio)[-1].lower()
  if ext == ".wav":
      y, sr = librosa.load(PATH+"Vodafone joy 845/"+audio, sr = 16000)
      y = y[0:20*sr]
      if data == "1":
         freq_rep = librosa.stft(y, n_fft=1023, win_length=625, hop_length=625, window='rect')
         ph = np.angle(freq_rep)
      else:
        freq_rep = librosa.stft(y, n_fft=1023, win_length=625, hop_length=625, window='rect')
        ph = np.angle(freq_rep)
        freq_rep = librosa.feature.melspectrogram(S=abs(freq_rep), n_mels=512, sr=16000, power=1.0)

      if data == "1":
        mag_db = librosa.amplitude_to_db(np.abs(freq_rep))
      else:
        mag_db = librosa.amplitude_to_db(freq_rep)  # mel

      # max and min values (in db) for conversion
      max_val = np.max(np.max(mag_db))
      min_val = np.min(np.min(mag_db))
      # converting into image
      freq_rep_img = np.round(((mag_db-min_val)/(max_val-min_val))*(255-0)+0)

      #create also masked image
      mask = np.ones([len(freq_rep_img), len(freq_rep_img[0])])
      mask[starting_point:(starting_point+m+1), :] = 0

      corrupted_img = mask * freq_rep_img

      composed = np.zeros([len(freq_rep_img), len(freq_rep_img[0])*2])
      composed[:,0:len(freq_rep_img[0])] = freq_rep_img
      composed[:,len(freq_rep_img[0]):len(freq_rep_img[0])*2] = corrupted_img

      composed_image = Image.fromarray(composed.astype(np.uint8))
      save_name = audio[:-4] #to eliminate '.wav' from file name  
      composed_image.save("frequency_images_test/speaker_test_"+str(id)+".png")
      id=id+1


In [ ]:
# Params
BUFFER_SIZE = 400
BATCH_SIZE = 1
IMG_WIDTH = 512
IMG_HEIGHT = 512

In [ ]:
# Load real and input(corrupted) image

def load(image_file):
  image = tf.io.read_file(image_file)
  image = tf.image.decode_png(image)
  w = tf.shape(image)[1]

  w = w // 2
  real_image = image[:, :w, :]
  input_image = image[:, w:, :]

  input_image = tf.cast(input_image, tf.float32)
  real_image = tf.cast(real_image, tf.float32)

  return input_image, real_image

# Functions

In [ ]:
def resize(input_image, real_image, height, width):
  input_image = tf.image.resize(input_image, [height, width],
                                method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)
  real_image = tf.image.resize(real_image, [height, width],
                               method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)

  return input_image, real_image

In [ ]:
def random_crop(input_image, real_image):
  stacked_image = tf.stack([input_image, real_image], axis=0)
  cropped_image = tf.image.random_crop(
      stacked_image, size=[2, IMG_HEIGHT, IMG_WIDTH, 3])

  return cropped_image[0], cropped_image[1]

In [ ]:
# normalizing the images to [-1, 1]
def normalize(input_image, real_image):
  input_image = (input_image / 127.5) - 1
  real_image = (real_image / 127.5) - 1

  return input_image, real_image

In [ ]:
@tf.function()
def random_jitter(input_image, real_image):
  # resizing to 286 x 286 x 3
  input_image, real_image = resize(input_image, real_image, IMG_HEIGHT, IMG_WIDTH)

  # randomly cropping to 256 x 256 x 3
  #input_image, real_image = random_crop(input_image, real_image)

  #if tf.random.uniform(()) > 0.5:
    # random mirroring
    #input_image = tf.image.flip_left_right(input_image)
    #real_image = tf.image.flip_left_right(real_image)

  return input_image, real_image

Random jittering does the following:
* Resize an image to a bigger height and width
* Randomly crop to target size
* Randomly flip the image horizontally

In [ ]:
def load_image_train(image_file):
  input_image, real_image = load(image_file)
  #input_image, real_image = random_jitter(input_image, real_image)
  input_image, real_image = normalize(input_image, real_image)

  return input_image, real_image

In [ ]:
def load_image_test(image_file):
  input_image, real_image = load(image_file)
  #input_image, real_image = resize(input_image, real_image, IMG_HEIGHT, IMG_WIDTH)
  input_image, real_image = normalize(input_image, real_image)

  return input_image, real_image

# Input Pipeline

In this example, we will use the [`tf.data`](https://www.tensorflow.org/guide/data) API, while it is not necessary to understand everything for the sake of this tutorial, check the link if you want to know more about how it works.

In a nutshell it is an API which allows you to handle data and feed them into tensorflow models.

In [ ]:
train_dataset = tf.data.Dataset.list_files('frequency_images_train/*.png')
train_dataset = train_dataset.map(load_image_train, num_parallel_calls=tf.data.experimental.AUTOTUNE)
train_dataset = train_dataset.shuffle(BUFFER_SIZE)
train_dataset = train_dataset.batch(BATCH_SIZE)

In [ ]:
test_dataset = tf.data.Dataset.list_files('frequency_images_test/*.png', shuffle=False)
test_dataset = test_dataset.map(load_image_test,deterministic=True)
test_dataset = test_dataset.batch(BATCH_SIZE)

In [ ]:
#import keras.backend as K
#test_dataset = tf.data.Dataset.list_files('frequency_images_test/*.png', shuffle=False)
#for f in test_dataset.take(4):
#  print(K.eval(f))


# Build the generator
  * Architecture is a modified [U-Net](https://arxiv.org/abs/1505.04597)
  * Each block in the encoder is (Conv -> [Batchnorm](https://arxiv.org/abs/1502.03167) -> [Leaky ReLU](https://ai.stanford.edu/~amaas/papers/relu_hybrid_icml2013_final.pdf))
  * There are [skip connections](https://en.wikipedia.org/wiki/Residual_neural_network) between the encoder and decoder (as in U-Net).

  ![unet_architecture](https://3qeqpr26caki16dnhd19sv6by6v-wpengine.netdna-ssl.com/wp-content/uploads/2019/05/Architecture-of-the-U-Net-Generator-Model.png)
  

In [ ]:
OUTPUT_CHANNELS = 1 # gray

## Downsampling layer: FILL THE CODE

In [ ]:
#FILL THE CODE: complete the downsampling layer that we will use to shrink the image input

def downsample(filters, size, apply_batchnorm=True):
  initializer = tf.random_normal_initializer(0., 0.02)

  result = tf.keras.Sequential()
  result.add(
#FILL THE CODE: Add a 2D convolutional layer, with the specified options, use strides=2, padding="same" and use_bias=False
      tf.keras.layers.Conv2D(filters, size, strides=2, padding='same',
                             kernel_initializer=initializer, use_bias=False))

  if apply_batchnorm:
    result.add(tf.keras.layers.BatchNormalization())

  result.add(tf.keras.layers.LeakyReLU())

  return result

## Upsampling Layer: FILL THE CODE

To upsample the images we are going to use a [`Conv2DTranspose`](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Conv2D) layer, which performs a [transposed convolution](https://towardsdatascience.com/what-is-transposed-convolutional-layer-40e5e6e31c11#:~:text=Transposed%20convolutions%20are%20standard%20convolutions,in%20a%20standard%20convolution%20operation.)

In [ ]:
#FILL THE CODE: complete the upsampling layer that we will increase image size after downsampling it

def upsample(filters, size, apply_dropout=False):
  initializer = tf.random_normal_initializer(0., 0.02)

  result = tf.keras.Sequential()
  result.add(
   #FILL THE CODE: Add a 2D transposed convolutional layer, with the specified options, use strides=2, padding='same' and use_bias=False 

    tf.keras.layers.Conv2DTranspose(filters, size, strides=2,
                                    padding='same',
                                    kernel_initializer=initializer,
                                    use_bias=False))

  result.add(tf.keras.layers.BatchNormalization())

  if apply_dropout:
      result.add(tf.keras.layers.Dropout(0.5))

  result.add(tf.keras.layers.ReLU())

  return result

In [ ]:
def Generator():
  inputs = tf.keras.layers.Input(shape=[512, 512, 1])

  down_stack = [
    downsample(32, 4, apply_batchnorm=False),
    downsample(64, 4), # (bs, 128, 128, 64)
    downsample(128, 4), # (bs, 64, 64, 128)
    downsample(256, 4), # (bs, 32, 32, 256)
    downsample(512, 4), # (bs, 16, 16, 512)
    downsample(512, 4), # (bs, 8, 8, 512)
    downsample(512, 4), # (bs, 4, 4, 512)
    downsample(512, 4), # (bs, 2, 2, 512)
    downsample(512, 4), # (bs, 1, 1, 512)
  ]

  up_stack = [
    upsample(512, 4, apply_dropout=True), # (bs, 2, 2, 1024)
    upsample(512, 4, apply_dropout=True), # (bs, 4, 4, 1024)
    upsample(512, 4, apply_dropout=True), # (bs, 8, 8, 1024)
    upsample(512, 4), # (bs, 16, 16, 1024)
    upsample(256, 4), # (bs, 32, 32, 512)
    upsample(128, 4), # (bs, 64, 64, 256)
    upsample(64, 4), # (bs, 128, 128, 128)
    upsample(32, 4)
  ]

  initializer = tf.random_normal_initializer(0., 0.02)
  last = tf.keras.layers.Conv2DTranspose(OUTPUT_CHANNELS, 4,
                                         strides=2,
                                         padding='same',
                                         kernel_initializer=initializer,
                                         activation='tanh') # (bs, 256, 256, 3)

  x = inputs

  # Downsampling through the model
  skips = []
  for down in down_stack:
    x = down(x)
    skips.append(x)

  skips = reversed(skips[:-1])

  # Upsampling and establishing the skip connections
  for up, skip in zip(up_stack, skips):
    x = up(x)
    x = tf.keras.layers.Concatenate()([x, skip])

  x = last(x)

  return tf.keras.Model(inputs=inputs, outputs=x)


In [ ]:
generator = Generator()
tf.keras.utils.plot_model(generator, show_shapes=True, dpi=64)

   ## Generator Loss: FILL THE CODE
    * sigmoid cross entropy loss of generated images and and array of ones (classification)
    * L1 loss -> Mean Absolute Error (MAE) between generated image and target image
      * this allows generated image to be structurally similar to target image
      * Formula to calculate total generator loss is:
        * $ \mathcal{L}_\text{gan} +\lambda \mathcal{L}_\text{L1} $ 
        
        with $\lambda = 100$ (decided by authors of the original paper)
     

```
# Questo è formattato come codice
```


The training procedure for the generator is shown below:



In [ ]:
LAMBDA = 100


In [ ]:
loss_object = tf.keras.losses.BinaryCrossentropy(from_logits=True)

In [ ]:
def generator_loss(disc_generated_output, gen_output, target):
  # FILL THE CODE:
  # define the GAN generator loss i.e. does the discriminator classify generator 
  # outputs as real?
  # HINT: use loss_object to compute the loss and tf.ones_like() to represent classification
  # result
  gan_loss = loss_object(tf.ones_like(disc_generated_output), disc_generated_output)


  # FILL THE CODE: mean absolute error
  # compute mean absolute error between target image and generated image
  # HINT: use tf.reduce_mean and tf.abs
  l1_loss = tf.reduce_mean(tf.abs(target - gen_output))

  # FILL THE CODE: compute generator loss as the sum of GAN loss and weighted
  # L1 loss
  total_gen_loss = gan_loss + (LAMBDA * l1_loss)

  return total_gen_loss, gan_loss, l1_loss

![Generator Update Image](https://github.com/tensorflow/docs/blob/master/site/en/tutorials/generative/images/gen.png?raw=1)

# Build the Discriminator
Discriminator is a PatchGAN
  * Each block is (Conv -> BatchNorm -> LeakyRelu)
  * shape of output after last layer is (bs,30,30,1)
    * N.B.-> each 30x30 patch of the output classifies a 70x70 portion of the input (PatchGAN)
  * Discriminator receives two inputs.
    * Input_1 image and target image, which it should classify as **real**.
    * Input_2 image and generated image, which it should classify as **fake**.
    * We concatenate Input_1 and Input_2 together in the code (```# tf.concat([inp, tar], axis=-1)```)




In [ ]:
def Discriminator():
  initializer = tf.random_normal_initializer(0., 0.02)

  inp = tf.keras.layers.Input(shape=[512, 512, 1], name='input_image')
  tar = tf.keras.layers.Input(shape=[512, 512, 1], name='target_image')

  x = tf.keras.layers.concatenate([inp, tar]) # (bs, 256, 256, channels*2)

  down1 = downsample(64, 4, False)(x) # (bs, 128, 128, 64)
  down2 = downsample(128, 4)(down1) # (bs, 64, 64, 128)
  down3 = downsample(256, 4)(down2) # (bs, 32, 32, 256)
  down4 = downsample(512, 4)(down3) # (bs, 16, 16, 512)

  zero_pad1 = tf.keras.layers.ZeroPadding2D()(down4) # (bs, 34, 34, 256) ORIGINALLY WAS ON DOWN3!
  conv = tf.keras.layers.Conv2D(512, 4, strides=1,
                                kernel_initializer=initializer,
                                use_bias=False)(zero_pad1) # (bs, 31, 31, 512)

  batchnorm1 = tf.keras.layers.BatchNormalization()(conv)

  leaky_relu = tf.keras.layers.LeakyReLU()(batchnorm1)

  zero_pad2 = tf.keras.layers.ZeroPadding2D()(leaky_relu) # (bs, 33, 33, 512)

  last = tf.keras.layers.Conv2D(1, 4, strides=1,
                                kernel_initializer=initializer)(zero_pad2) # (bs, 30, 30, 1)

  return tf.keras.Model(inputs=[inp, tar], outputs=last)

In [ ]:
discriminator = Discriminator()
tf.keras.utils.plot_model(discriminator, show_shapes=True, dpi=64)

## Discriminator Loss: FILL THE CODE
The discriminator loss function takes 2 inputs:
* real images
  * real_loss is a sigmoid cross entropy loss of the real images and an array of ones(since these are the real images)
* generated images
  * generated_loss is a sigmoid cross entropy loss of the generated images and an array of zeros(since these are the fake images)
* Total_loss is a sum of real_loss and generated_loss

In [ ]:
def discriminator_loss(disc_real_output, disc_generated_output):
  # FILL THE CODE:
  # define the GAN discriminator loss remember ideally discriminator should classify
  # as 0 fake images and as 1 real images
  # HINT: use loss_object and tf.ones_like and tf.zeros_like

  # loss related to real images
  real_loss = loss_object(tf.ones_like(disc_real_output), disc_real_output)

  # loss related to fake images
  generated_loss = loss_object(tf.zeros_like(disc_generated_output), disc_generated_output)

  # FILL THE CODE:
  # compute output loss as the sum of real and generated loss
  total_disc_loss = real_loss + generated_loss

  return total_disc_loss

# Define the Optimizers and Checkpoint-saver

In [ ]:
generator_optimizer = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)
discriminator_optimizer = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)

In [ ]:
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(generator_optimizer=generator_optimizer,
                                 discriminator_optimizer=discriminator_optimizer,
                                 generator=generator,
                                 discriminator=discriminator)

# Generate Images

Write a function to plot some images during training:
  * We pass images from the test dataset to the generator
  * The generator will then translate the input image into the output
  * Last step is to plot the predictions

In [ ]:
def generate_images(model, test_input, tar):
  prediction = model(test_input, training=True)
  plt.figure(figsize=(15,15))

  display_list = [test_input[0], tar[0], prediction[0]]
  title = ['Input Image', 'Ground Truth', 'Predicted Image']

  for i in range(3):
    plt.subplot(1, 3, i+1)
    plt.title(title[i])
    # getting the pixel values between [0, 1] to plot it.
    img_to_print = display_list[i].numpy()
    img_to_print = img_to_print.squeeze() # to adapt it for imshow
    plt.imshow(img_to_print)
    plt.axis('off')
  plt.show()

  return display_list[2].numpy(), display_list[1].numpy() #returning the predicted image and the original

In [ ]:
for example_input, example_target in test_dataset.take(1):
  generate_images(generator, example_input, example_target)

# Training: FILL THE CODE
* For each example input generate an output
* The discriminator receives the input_image and the generated image as the first input. The second input is the input_image and the target_image.
* Calculate Generator_loss and discriminator_loss
* Then, we calculate the gradients of loss with respect to both the generator and the discriminator variables(inputs) and apply those to the optimizer.
* Then log the losses to TensorBoard.


In [ ]:
EPOCHS = 5

In [ ]:
import datetime
log_dir="logs/"

summary_writer = tf.summary.create_file_writer(
  log_dir + "fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))

In [ ]:
@tf.function
def train_step(input_image, target, epoch):

  # Use tf.gradient tape to keep track of the generator and discriminator gradients
  with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:

    # FILL THE code: compute the output of the generator (i.e. fake images)   
    gen_output = generator(input_image, training=True)

    # FILL THE CODE: compute the output of the discriminator w.r.t. real images 
    disc_real_output = discriminator([input_image, target], training=True)
    # FILL THE CODE: compute the output of the discriminator w.r.t. fake images 
    disc_generated_output = discriminator([input_image, gen_output], training=True)

    # compute the 2 losses
    gen_total_loss, gen_gan_loss, gen_l1_loss = generator_loss(disc_generated_output, gen_output, target)
    disc_loss = discriminator_loss(disc_real_output, disc_generated_output)

  # FILL THE CODE: 
  # Compute the gradient of the generator loss w.r.t. the generator parameters 
  # HINT: gen_tape.gradient(...), disc_tape.gradient(...)   
  generator_gradients = gen_tape.gradient(gen_total_loss,
                                          generator.trainable_variables)
    # Compute the gradient of the discriminator loss w.r.t. the discriminator parameters
  discriminator_gradients = disc_tape.gradient(disc_loss,
                                               discriminator.trainable_variables)

  # Apply the gradient to the optimizer so that it can update the model accordingly  
  generator_optimizer.apply_gradients(zip(generator_gradients,
                                          generator.trainable_variables))
  discriminator_optimizer.apply_gradients(zip(discriminator_gradients,
                                              discriminator.trainable_variables))

  with summary_writer.as_default():
    tf.summary.scalar('gen_total_loss', gen_total_loss, step=epoch)
    tf.summary.scalar('gen_gan_loss', gen_gan_loss, step=epoch)
    tf.summary.scalar('gen_l1_loss', gen_l1_loss, step=epoch)
    tf.summary.scalar('disc_loss', disc_loss, step=epoch)

Actual training loop:

* Iterates over the number of epochs.

* On each epoch it clears the display, and runs generate_images to show it's progress.
* On each epoch it iterates over the training dataset, printing a '.' for each example.
* It saves a checkpoint every 20 epochs.

In [ ]:
def fit(train_ds, epochs, test_ds):
  for epoch in range(epochs):
    start = time.time()

    display.clear_output(wait=True)

    for example_input, example_target in test_ds.take(1):
      generate_images(generator, example_input, example_target)
    print("Epoch: ", epoch)

    # Train
    for n, (input_image, target) in train_ds.enumerate():
      print('.', end='')
      if (n+1) % 100 == 0:
        print()
      train_step(input_image, target, epoch)
    print()

    # saving (checkpoint) the model every 20 epochs
    if (epoch + 1) % 20 == 0:
      checkpoint.save(file_prefix = checkpoint_prefix)

    print ('Time taken for epoch {} is {} sec\n'.format(epoch + 1,
                                                        time.time()-start))
  checkpoint.save(file_prefix = checkpoint_prefix)

This training loop saves logs you can easily view in TensorBoard to monitor the training progress. Working locally you would launch a separate tensorboard process. In a notebook, if you want to monitor with TensorBoard it's easiest to launch the viewer before starting the training.

To launch the viewer paste the following into a code-cell:

In [ ]:
% load_ext tensorboard
% tensorboard --logdir {log_dir}

Run the training loop:


In [ ]:
fit(train_dataset, EPOCHS, test_dataset)

In [ ]:
display.IFrame(
    src="https://tensorboard.dev/experiment/lZ0C6FONROaUMfjYkVyJqw",
    width="100%",
    height="1000px")

Interpreting the logs from a GAN is more subtle than a simple classification or regression model. Things to look for::

* Check that neither model has "won". If either the gen_gan_loss or the disc_loss gets very low it's an indicator that this model is dominating the other, and you are not successfully training the combined model.

* The value log(2) = 0.69 is a good reference point for these losses, as it indicates a perplexity of 2: That the discriminator is on average equally uncertain about the two options.

* For the disc_loss a value below 0.69 means the discriminator is doing better than random, on the combined set of real+generated images.

*For the gen_gan_loss a value below 0.69 means the generator i doing better than random at foolding the descriminator.

*As training progresses the gen_l1_loss should go down

# Restore the latest checkpoint and test

In [ ]:
!ls {checkpoint_dir}

In [ ]:
# Run the trained model on a few examples from the test dataset
for inp, tar in test_dataset.take(5):
  out = generate_images(generator, inp, tar)

# Take generated images and go back to audio

In [ ]:
def convert(freq_img, ph, data, reconstruction, max_val, min_val, sr):
  inpainted_freq_rep_db = ((freq_img-0)/(255-0))*(max_val-min_val)+min_val
  inpainted_freq_rep = librosa.db_to_amplitude(inpainted_freq_rep_db)

  if data == "1": #stft
    if reconstruction == "1": #original phase
      #inpainted_freq_rep = librosa.db_to_amplitude(inpainted_freq_rep_db)
      real = inpainted_freq_rep * np.cos(ph)
      imag = inpainted_freq_rep * np.sin(ph)
      mod_freq_rep = np.zeros([len(real), len(real[0])], dtype=np.complex_)
      # inverse transform (creating a complex stft from modified magnitude and original phase)
      for i in range(len(real)):
        for j in range(len(real[0])):
          mod_freq_rep[i, j] = complex(real[i, j], imag[i, j])
          
      output = librosa.istft(mod_freq_rep, win_length=625, hop_length=625, window='rect')

    else: #griffinlim
      #inpainted_freq_rep = librosa.db_to_amplitude(inpainted_freq_rep_db)
      output = librosa.griffinlim(inpainted_freq_rep, win_length=625, hop_length=625, window='rect')

  else: #log mel
    if reconstruction == "1": #original phase
      #inpainted_freq_rep = librosa.db_to_power(inpainted_freq_rep_db)
      inpainted_freq_rep = librosa.feature.inverse.mel_to_stft(inpainted_freq_rep, sr, n_fft=1023, power=1.0)
      real = inpainted_freq_rep * np.cos(ph)
      imag = inpainted_freq_rep * np.sin(ph)
      mod_freq_rep = np.zeros([len(real), len(real[0])], dtype=np.complex_)
      # inverse transform (creating a complex stft from modified magnitude and original phase)
      for i in range(len(real)):
        for j in range(len(real[0])):
          mod_freq_rep[i, j] = complex(real[i, j], imag[i, j])
      
      output = librosa.istft(mod_freq_rep, win_length=625, hop_length=625, window='rect')
      
    else: #griffinlim
      #inpainted_freq_rep = librosa.db_to_power(inpainted_freq_rep_db)
      mod_freq_rep = librosa.feature.inverse.mel_to_stft(inpainted_freq_rep, sr, n_fft=1023,power=1.0)
      output = librosa.griffinlim(mod_freq_rep, win_length=625, hop_length=625, window='rect')
    
  return output

In [ ]:
from math import log10, sqrt

def wada_snr(wav):
    # Direct blind estimation of the SNR of a speech signal.
    #
    # Paper on WADA SNR:
    #   http://www.cs.cmu.edu/~robust/Papers/KimSternIS08.pdf
    #
    # This function was adapted from this matlab code:
    #   https://labrosa.ee.columbia.edu/projects/snreval/#9
    #
    # MIT license, John Meade, 2020

    # init
    eps = 1e-10
    # next 2 lines define a fancy curve derived from a gamma distribution -- see paper
    db_vals = np.arange(-20, 101)
    g_vals = np.array([0.40974774, 0.40986926, 0.40998566, 0.40969089, 0.40986186, 0.40999006, 0.41027138, 0.41052627, 0.41101024, 0.41143264, 0.41231718, 0.41337272, 0.41526426, 0.4178192 , 0.42077252, 0.42452799, 0.42918886, 0.43510373, 0.44234195, 0.45161485, 0.46221153, 0.47491647, 0.48883809, 0.50509236, 0.52353709, 0.54372088, 0.56532427, 0.58847532, 0.61346212, 0.63954496, 0.66750818, 0.69583724, 0.72454762, 0.75414799, 0.78323148, 0.81240985, 0.84219775, 0.87166406, 0.90030504, 0.92880418, 0.95655449, 0.9835349 , 1.01047155, 1.0362095 , 1.06136425, 1.08579312, 1.1094819 , 1.13277995, 1.15472826, 1.17627308, 1.19703503, 1.21671694, 1.23535898, 1.25364313, 1.27103891, 1.28718029, 1.30302865, 1.31839527, 1.33294817, 1.34700935, 1.3605727 , 1.37345513, 1.38577122, 1.39733504, 1.40856397, 1.41959619, 1.42983624, 1.43958467, 1.44902176, 1.45804831, 1.46669568, 1.47486938, 1.48269965, 1.49034339, 1.49748214, 1.50435106, 1.51076426, 1.51698915, 1.5229097 , 1.528578  , 1.53389835, 1.5391211 , 1.5439065 , 1.54858517, 1.55310776, 1.55744391, 1.56164927, 1.56566348, 1.56938671, 1.57307767, 1.57654764, 1.57980083, 1.58304129, 1.58602496, 1.58880681, 1.59162477, 1.5941969 , 1.59693155, 1.599446  , 1.60185011, 1.60408668, 1.60627134, 1.60826199, 1.61004547, 1.61192472, 1.61369656, 1.61534074, 1.61688905, 1.61838916, 1.61985374, 1.62135878, 1.62268119, 1.62390423, 1.62513143, 1.62632463, 1.6274027 , 1.62842767, 1.62945532, 1.6303307 , 1.63128026, 1.63204102])

    # peak normalize, get magnitude, clip lower bound
    wav = np.array(wav)
    wav = wav / np.max(np.abs(wav))
    abs_wav = np.abs(wav)
    abs_wav[abs_wav < eps] = eps

    # calcuate statistics
    # E[|z|]
    v1 = max(eps, abs_wav.mean())
    # E[log|z|]
    v2 = np.log(abs_wav).mean()
    # log(E[|z|]) - E[log(|z|)]
    v3 = np.log(v1) - v2

    # table interpolation
    wav_snr_idx = None
    if any(g_vals < v3):
        wav_snr_idx = np.where(g_vals < v3)[0].max()
    # handle edge cases or interpolate
    if wav_snr_idx is None:
        wav_snr = db_vals[0]
    elif wav_snr_idx == len(db_vals) - 1:
        wav_snr = db_vals[-1]
    else:
        wav_snr = db_vals[wav_snr_idx] + \
            (v3-g_vals[wav_snr_idx]) / (g_vals[wav_snr_idx+1] - \
            g_vals[wav_snr_idx]) * (db_vals[wav_snr_idx+1] - db_vals[wav_snr_idx])

    # Calculate SNR
    dEng = sum(wav**2)
    dFactor = 10**(wav_snr / 10)
    dNoiseEng = dEng / (1 + dFactor) # Noise energy
    dSigEng = dEng * dFactor / (1 + dFactor) # Signal energy
    snr = 10 * np.log10(dSigEng / dNoiseEng)

    return snr

def PSNR(original, compressed):
  mse = np.mean((original - compressed) ** 2)
  if mse == 0:  # MSE is zero means no noise is present in the signal .
  # Therefore PSNR have no importance.
    return 100
  max_pixel = 255.0
  psnr = 20 * log10(max_pixel / sqrt(mse))
  return psnr

def alternative_SNR(y):

  def energy(x):
    e = np.sum(x ** 2)
    return e

  Z = librosa.stft(y, n_fft=1023, win_length=625, hop_length=625, window='rect')
  z = librosa.istft(Z, win_length=625, hop_length=625, window='rect')
  n = y[:z.size] - z
  snr = 10 * np.log10(energy(y)/energy(n))
  return snr


In [ ]:
!pip install pesq

In [ ]:
import cv2
from scipy.io import wavfile
import csv
from google.colab import files
from skimage.metrics import structural_similarity as ssim
from pesq import pesq

reconstruction = "2"

iteration = round(m/2)+1
n_files = 24
file_list = sorted(os.listdir('MOBIPHONE/Vodafone joy 845/'))
image_list = sorted(os.listdir('frequency_images_test/'))
input_indices = ["1","10","11","12","13","14","15","16","17","18","19","2","20","21","21","22","23","24","3","4","5","6","7","8","9"]
#THE CORRESPONDANCE IS (ex) : speaker_10.png --> speaker18.wav --> output_2.wav and so on...

i = 0
# delete old csv if present
if os.path.isfile('pix_2_pix_results_'+str(data)+'_'+str(reconstruction)+'_'+str(iteration)+'_.csv'):
  os.remove('pix_2_pix_results_'+str(data)+'_'+str(reconstruction)+'_'+str(iteration)+'_.csv')

# creating csv file where to save results (SNR/PSNR)
with open('pix_2_pix_results_'+str(data)+'_'+str(reconstruction)+'_'+str(iteration)+'_.csv', mode='w') as pix_2_pix: # OPENING FILE (TILL THE END)
  pix_2_pix_writer = csv.writer(pix_2_pix, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
  pix_2_pix_writer.writerow(['FILE', 'SNR', 'PSNR', 'SSIM', 'PESQ'])
  #so, for example...model_based_results_1_1.csv means results of data = 1 (stft) and reconstruction = 1 (original phase)
  for inp, tar in test_dataset.take(n_files):
    print("\nFILE "+str(i+1)+"...")
    out, orig = generate_images(generator, inp, tar)
        
    #recovering phase
    index = int(input_indices[i])
    audio_to_load = file_list[index-1]
    y, sr = librosa.load('MOBIPHONE/Vodafone joy 845/'+audio_to_load, sr = 16000)
    y = y[0:20*sr]
    if data == "1":
      freq_rep = librosa.stft(y, n_fft=1023, win_length=625, hop_length=625, window='rect')
      ph = np.angle(freq_rep)
    else:
      freq_rep = librosa.stft(y, n_fft=1023, win_length=625, hop_length=625, window='rect')
      ph = np.angle(freq_rep)
      freq_rep = librosa.feature.melspectrogram(S=abs(freq_rep), n_mels=512, sr=16000, power=1.0)

    # recovering also original image (for PSNR)
    if data == "1":
      mag_db = librosa.amplitude_to_db(np.abs(freq_rep))
    else:
      mag_db = librosa.amplitude_to_db(freq_rep)  # mel

    # max and min values (in db) for conversion
    max_val = np.max(np.max(mag_db))
    min_val = np.min(np.min(mag_db))
    # converting into image
    img_np = np.round(((mag_db-min_val)/(max_val-min_val))*(255-0)+0)

    out_np = out #will be the last inpainted (predicted) image
    orig_np = orig #will be the original image

    # converting into 0,255
    max_val_orig = np.max(np.max(orig_np[:,:,0]))
    min_val_orig = np.min(np.min(orig_np[:,:,0]))
    max_o = np.max(np.max(out_np[:,:,0]))
    min_o = np.min(np.min(out_np[:,:,0]))
    # converting into image
    orig_np = np.round(((orig_np[:,:,0]-min_val_orig)/(max_val_orig-min_val_orig))*(255-0)+0)
    out_np = np.round(((out_np[:,:,0] - min_o) / (max_o - min_o)) * (255 - 0) + 0)
      
    #PSNR
    psnr = PSNR(orig_np[:,:], out_np[:,:])
        
    #resizing to original size
    #out_np = cv2.resize(out_np, dsize=(len(freq_rep_img[0]), len(freq_rep_img)), interpolation=cv2.INTER_CUBIC)
        
    min_val_out = np.min(np.min(out_np[:,:]))
    max_val_out = np.max(np.max(out_np[:,:]))

    # here I need to calculate the original phase
    output = convert(out_np[:,:], ph, data, reconstruction, max_val, min_val, sr)
    min_out = np.min(output)
    max_out = np.max(output)
    output=((output-min_out)/(max_out-min_out))*(max(y)-min(y))+min(y) # converting back
        
    # writing output
    wavfile.write('output_'+str(i+1)+'.wav', sr, output.astype(np.float32))

    # SNR
    o, sr = librosa.load('output_'+str(i+1)+'.wav',sr=16000)
    snr = alternative_SNR(o)
          
    #PESQ
    psq = pesq(16000, y, o, 'wb')

    # SSIM
    images_ssim = ssim(orig_np[:,:], out_np[:,:], data_range=orig_np[:,:].max() - out_np[:,:].min())

    # save results in csv
    pix_2_pix_writer.writerow([os.path.basename(str('MOBIPHONE/Vodafone joy 845/'+str(audio_to_load[:-4]))), str(snr), str(psnr), str(images_ssim), str(psq)])

    i = i + 1


# download csv
files.download('pix_2_pix_results_'+str(data)+'_'+str(reconstruction)+'_'+str(iteration)+'_.csv')

##### Copyright 2019 The TensorFlow Authors.

Licensed under the Apache License, Version 2.0 (the "License");

